In [1]:
import sys
sys.path.append('../')

from model import CNN
from utils import accuracy

import nni
from nni.retiarii.oneshot.pytorch import DartsTrainer

import torch
from torchvision import transforms
from torchvision.datasets import FashionMNIST

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])
train_dataset = FashionMNIST(root="./data", train=True, download=True, transform=transform)
val_dataset = FashionMNIST(root="./data", train=False, download=True, transform=transform)

C:\Users\anton.sotnikov\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [10]:
model = CNN(input_size=1, 
            in_channels=1, 
            channels=2, 
            n_classes=10, 
            n_layers=3, 
            n_nodes=4,
            stem_multiplier=3,
            auxiliary=False)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters())

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [12]:
trainer = DartsTrainer(
    model=model,
    loss=criterion,
    metrics=lambda output, target: accuracy(output, target),
    optimizer=optimizer,
    num_epochs=1,
    dataset=train_dataset,
    batch_size=64,
    log_frequency=20,
    device=device
    )

trainer.fit()